In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model

In [2]:
def InceptionBlock(X, filters, training=True):
    
    # 1x1 convolution path
    X_1x1 = tfl.Conv2D(filters = filters[0], kernel_size = 1, strides = 1, padding = 'same')(X)
    X_1x1 = tfl.BatchNormalization(axis = 3)(X_1x1, training=training)
    X_1x1 = tfl.Activation('relu')(X_1x1)
    
    # 3x3 convolution path
    X_3x3 = tfl.Conv2D(filters = filters[1], kernel_size = 1, strides = 1, padding = 'same')(X)
    X_3x3 = tfl.BatchNormalization(axis = 3)(X_3x3, training=training)
    X_3x3 = tfl.Activation('relu')(X_3x3)
    
    X_3x3 = tfl.Conv2D(filters = filters[2], kernel_size = 3, strides = 1, padding = 'same')(X_3x3)
    X_3x3 = tfl.BatchNormalization(axis = 3)(X_3x3, training=training)
    X_3x3 = tfl.Activation('relu')(X_3x3)
    
    # 5x5 convolution path
    X_5x5 = tfl.Conv2D(filters = filters[3], kernel_size = 1, strides = 1, padding = 'same')(X)
    X_5x5 = tfl.BatchNormalization(axis = 3)(X_5x5, training=training)
    X_5x5 = tfl.Activation('relu')(X_5x5)
    
    X_5x5 = tfl.Conv2D(filters = filters[4], kernel_size = 5, strides = 1, padding = 'same')(X_5x5)
    X_5x5 = tfl.BatchNormalization(axis = 3)(X_5x5, training=training)
    X_5x5 = tfl.Activation('relu')(X_5x5)
    
    # maxPooling path
    X_pool = tfl.MaxPooling2D(pool_size = (3, 3), strides= 1, padding ='same')(X)
    
    X_pool = tfl.Conv2D(filters = filters[5], kernel_size = 1, strides = 1, padding = 'same')(X_pool)
    X_pool = tfl.BatchNormalization(axis = 3)(X_pool, training=training)
    X_pool = tfl.Activation('relu')(X_pool)
    
    X_out = tfl.concatenate(inputs = [X_1x1, X_3x3, X_5x5, X_pool], axis = 3)
    
    return X_out   
    

In [3]:
def OutputBlock(X, classes, training=True):
    
    X = tfl.AveragePooling2D(pool_size = 5, strides = 3, padding = 'same')(X)
    
    X = tfl.Conv2D(filters = 128, kernel_size = 1, strides = 1, padding = 'same')(X)
    X = tfl.BatchNormalization(axis = 3)(X, training=training)
    X = tfl.Activation('relu')(X)
    
    X = tfl.Flatten()(X)
    X = tfl.Dense(units = 1024, activation = "relu")(X)
    
    X = tfl.Dense(units = classes, activation='softmax')(X)
    
    return X    
    

In [4]:
def GoogleNet(input_shape, classes):
    
    X_input = tfl.Input(input_shape)
    
    X = tfl.Conv2D(filters = 64, kernel_size = 7, strides = 2, padding = 'same')(X_input)
    X = tfl.BatchNormalization(axis = 3)(X)
    X = tfl.Activation('relu')(X)
    
    X = tfl.MaxPooling2D(pool_size = (3, 3), strides= 2, padding ='same')(X)
    
    X = tfl.Conv2D(filters = 64, kernel_size = 1, strides = 1, padding = 'same')(X)
    X = tfl.BatchNormalization(axis = 3)(X)
    X = tfl.Activation('relu')(X)
    
    X = tfl.Conv2D(filters = 192, kernel_size = 3, strides = 1, padding = 'same')(X)
    X = tfl.BatchNormalization(axis = 3)(X)
    X = tfl.Activation('relu')(X)
    
    X = tfl.MaxPooling2D(pool_size = (3, 3), strides= 2, padding ='same')(X)
    
    X = InceptionBlock(X, filters =[64, 96, 128, 16, 32, 32])
    
    X = InceptionBlock(X, filters =[128, 128, 192, 32, 96, 64])
    
    X = tfl.MaxPooling2D(pool_size = (3, 3), strides= 2, padding ='same')(X)
    
    X = InceptionBlock(X, filters =[192, 96, 208, 16, 48, 64])
    
    X = InceptionBlock(X, filters =[128, 128, 192, 32, 96, 64])
    
    X_out1 = OutputBlock(X, classes = classes)
    
    X = InceptionBlock(X, filters =[160, 112, 224, 24, 64, 64])
    
    X = InceptionBlock(X, filters =[128, 128, 256, 24, 64, 64])
    
    X = InceptionBlock(X, filters =[112, 144, 288, 32, 64, 64])
    
    X_out2 = OutputBlock(X, classes = classes)
    
    X = InceptionBlock(X, filters =[256, 160, 320, 32, 128, 128])
    
    X = tfl.MaxPooling2D(pool_size = (3, 3), strides= 2, padding ='same')(X)
    
    X = InceptionBlock(X, filters =[256, 160, 320, 32, 128, 128])
    
    X = InceptionBlock(X, filters =[384, 192, 384, 48, 128, 128])
    
    X = tfl.AveragePooling2D(pool_size = 7, strides = 1, padding = 'same')(X)
    
    X_out3 = tfl.Dense(units = classes, activation='softmax')(X)
    
    model = tf.keras.Model(inputs = X_input, outputs = [X_out1, X_out2, X_out3], name = 'GoogleNet-V1')
    
    return model   
    

In [5]:
model = GoogleNet(input_shape = (128, 128, 3), classes = 1000)
print(model.summary())

Model: "GoogleNet-V1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 64)   0           batch_normalization[0][0]        
_______________________________________________________________________________________